In [1]:
import pandas as pd
import sklearn as skl
import plotly
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras_tuner as kt
import psycopg2

In [2]:
gdpwb_df = pd.read_csv('gdpwb.csv')

In [3]:
medals_df = pd.read_csv('olympic_medals.csv')

In [4]:
medals_df.head()

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/fatima-galvez,Fatima GALVEZ,Spain,ES,ESP
1,Shooting,tokyo-2020,Trap Mixed Team,Mixed,GOLD,GameTeam,Spain,https://olympics.com/en/athletes/alberto-ferna...,Alberto FERNANDEZ,Spain,ES,ESP
2,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/alessandra-pe...,Alessandra PERILLI,San Marino,SM,SMR
3,Shooting,tokyo-2020,Trap Mixed Team,Mixed,SILVER,GameTeam,San Marino,https://olympics.com/en/athletes/gian-marco-berti,Gian Marco BERTI,San Marino,SM,SMR
4,Shooting,tokyo-2020,Trap Mixed Team,Mixed,BRONZE,GameTeam,United States of America,https://olympics.com/en/athletes/madelynn-ann-...,Madelynn Ann BERNAU,United States of America,US,USA


In [5]:
gdpwb_df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', 'Unnamed: 65'],
      dtype='object')

In [6]:
gdpwb_df = gdpwb_df.drop(['Indicator Name', 'Indicator Code'], 1)

C:\Users\15129\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [7]:
gdpwb_df.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,1.931311e+10,1.972349e+10,2.149392e+10,2.573321e+10,2.352744e+10,2.681057e+10,2.915216e+10,3.017317e+10,...,9.505214e+11,9.642424e+11,9.848071e+11,9.199300e+11,8.733549e+11,9.853557e+11,1.012853e+12,1.009910e+12,9.207923e+11,NaN
2,Afghanistan,AFG,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,1.400000e+09,1.673333e+09,...,1.990732e+10,2.014640e+10,2.049713e+10,1.913421e+10,1.811656e+10,1.875347e+10,1.805323e+10,1.879945e+10,2.011614e+10,NaN
3,Africa Western and Central,AFW,1.040428e+10,1.112805e+10,1.194335e+10,1.267652e+10,1.383858e+10,1.486247e+10,1.583285e+10,1.442643e+10,...,7.275714e+11,8.207876e+11,8.649666e+11,7.607297e+11,6.905430e+11,6.837416e+11,7.416916e+11,7.945725e+11,7.845876e+11,NaN
4,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.941719e+10,5.837598e+10,NaN


In [8]:
gdpwb_melt = gdpwb_df.melt(id_vars=['Country Name','Country Code'], 
             value_vars=['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
                var_name='year',
                value_name='gdp')

In [ ]:
gdpwb_melt[gdpwb_melt['Country Name']=='Aruba']

In [9]:
gdpwb_df.columns

Index(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', 'Unnamed: 65'],
      dtype='object')

In [10]:
pop_df = pd.read_csv('population.csv', skiprows = 4)

In [11]:
pop_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,NaN


In [14]:
pop_df.columns

Index(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', 'Unnamed: 65'],
      dtype='object')

In [16]:
pop_df.head()

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,57702.0,58044.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,NaN
1,Africa Eastern and Southern,AFE,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,152752671.0,156876454.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,NaN
2,Afghanistan,AFG,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,10174840.0,10399936.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,NaN
3,Africa Western and Central,AFW,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,109701811.0,112195950.0,...,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,NaN
4,Angola,AGO,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,5781305.0,5774440.0,...,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,NaN


In [17]:
pop_melt = pop_df.melt(id_vars=['Country Name','Country Code'], 
             value_vars=['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020'],
                var_name='year',
                value_name='population')

In [18]:
gdp_pop_df = pd.merge(pop_melt, gdpwb_melt,
                        on=  ['Country Name','Country Code', 'year'],
                        how = 'left')
gdp_pop_df.head(100)

,Country Name,Country Code,year,population,gdp
0,Aruba,ABW,1960,54208.0,NaN
1,Africa Eastern and Southern,AFE,1960,130836765.0,1.931311e+10
2,Afghanistan,AFG,1960,8996967.0,5.377778e+08
3,Africa Western and Central,AFW,1960,96396419.0,1.040428e+10
4,Angola,AGO,1960,5454938.0,NaN
...,...,...,...,...,...
95,High income,HIC,1960,759783611.0,1.065825e+12
96,"Hong Kong SAR, China",HKG,1960,3075605.0,1.320797e+09
97,Honduras,HND,1960,2038636.0,3.356500e+08
98,Heavily indebted poor countries (HIPC),HPC,1960,161734348.0,1.741892e+10


In [19]:
hosts_df = pd.read_csv('hosts_cleaned.csv')
hosts_df.head()

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,tokyo-2020,2021-08-08,2021-07-23,Japan,Tokyo 2020,Summer,2020
1,pyeongchang-2018,2018-02-25,2018-02-08,Republic of Korea,PyeongChang 2018,Winter,2018
2,rio-2016,2016-08-21,2016-08-05,Brazil,Rio 2016,Summer,2016
3,sochi-2014,2014-02-23,2014-02-07,Russian Federation,Sochi 2014,Winter,2014
4,london-2012,2012-08-12,2012-07-27,Great Britain,London 2012,Summer,2012


In [ ]:
#medals_gdp_pop = pd.merge(hosts_df, gdp_pop_df,
                        #left_on=  ['game_location'],
                        #right_on= ['Country Name'],
                        #how = 'left')
#gdp_merged_df.head()

In [20]:
olympics_results = pd.merge(hosts_df, medals_df,
                           left_on=  ['game_slug'],
                           right_on= ['slug_game'],
                           how = 'left')
olympics_results.head(10)
olympics_results.columns

Index(['game_slug', 'game_end_date', 'game_start_date', 'game_location',
       'game_name', 'game_season', 'game_year', 'discipline_title',
       'slug_game', 'event_title', 'event_gender', 'medal_type',
       'participant_type', 'participant_title', 'athlete_url',
       'athlete_full_name', 'country_name', 'country_code',
       'country_3_letter_code'],
      dtype='object')

In [22]:
#clean_olympics_results= olympics_results.drop(['game_slug', 'game_end_date', 'game_start_date','game_name',  'slug_game', 'event_title', 'participant_type',  'athlete_url','athlete_full_name', 'country_code','country_3_letter_code'], 1)
clean_olympics_results.columns

Index(['game_location', 'game_season', 'game_year', 'discipline_title',
       'event_gender', 'medal_type', 'participant_title', 'country_name'],
      dtype='object')

In [23]:
olympic_wb = pd.merge(clean_olympics_results, gdp_pop_df,
                        left_on=  ['game_location'],
                        right_on= ['Country Name'],
                        how = 'left')
olympic_wb.head()

,game_location,game_season,game_year,discipline_title,event_gender,medal_type,participant_title,country_name,Country Name,Country Code,year,population,gdp
0,Japan,Summer,2020,Shooting,Mixed,GOLD,Spain,Spain,Japan,JPN,1960,93216000.0,4.430734e+10
1,Japan,Summer,2020,Shooting,Mixed,GOLD,Spain,Spain,Japan,JPN,1961,94055000.0,5.350862e+10
2,Japan,Summer,2020,Shooting,Mixed,GOLD,Spain,Spain,Japan,JPN,1962,94933000.0,6.072302e+10
3,Japan,Summer,2020,Shooting,Mixed,GOLD,Spain,Spain,Japan,JPN,1963,95900000.0,6.949813e+10
4,Japan,Summer,2020,Shooting,Mixed,GOLD,Spain,Spain,Japan,JPN,1964,96903000.0,8.174901e+10


In [25]:
olympic_wb.shape

(1008430, 13)